In [1]:
import pandas as pd
import numpy as np
import os
from scipy.sparse import coo_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/DMC

Mounted at /content/drive
/content/drive/My Drive/DMC


# preprocessing transactions.csv

1. remove transaction records that only contain one transaction in one session
2. recommended_books: a list to be recommended. 
3. add 2 new columns to transaction df
  - session_count: how many transaction records in the session
  - operate: sum click, basket, order data by weights (need to be tuned by evaluation)

In [3]:
evaluation_books = pd.read_csv('evaluation.csv')
items = pd.read_csv('items.csv', sep='|')
items = items.set_index('itemID')

transaction_original = pd.read_csv('transactions.csv', sep='|')
session_counts = transaction_original['sessionID'].value_counts()
transaction_original['session_count'] = transaction_original['sessionID'].apply(lambda x: session_counts[x])
print('there are %d transaction records in transactions' % transaction_original.shape[0])
print('there are %d recommended books in transactions' % len(set(evaluation_books['itemID']).intersection(set(
    transaction_original.index.values))))
transaction_original = transaction_original[transaction_original['session_count']>1]  # remove those transactions that only operate 1 book
recommended_books = set(evaluation_books['itemID']).intersection(set(transaction_original.index.values))
print('after removing transactions without only one session, there are %d records in transactions, %d recommend books out of 1000 books'
      % (transaction_original.shape[0], len(recommended_books)))
w_click = 0.1
w_basket = 0.3
w_order = 0.6
transaction_original['operate'] = (w_click * transaction_original['click'] + 
                                   w_basket * transaction_original['basket'] + 
                                   w_order * transaction_original['order'])

display(items.head())
display(evaluation_books.head())
display(transaction_original.head())

there are 365143 transaction records in transactions
there are 1000 recommended books in transactions
after removing transactions without only one session, there are 129501 records in transactions, 364 recommend books out of 1000 books


,title,author,publisher,main topic,subtopics
itemID,,,,,
21310,Princess Poppy: The Big Mix Up,Janey Louise Jones,Penguin Random House Children's UK,YFB,[5AH]
73018,Einfach zeichnen! Step by Step,Wiebke Krabbe,Schwager und Steinlein,AGZ,"[5AJ,AGZ,WFA,YBG,YBL,YNA,YPA]"
19194,Red Queen 1,Victoria Aveyard,Orion Publishing Group,YFH,"[5AP,FBA]"
40250,Meine Kindergarten-Freunde (Pirat),NaN,Ars Edition GmbH,YB,"[5AC,5AD,YBG,YBL,YF]"
46107,Mein großes Schablonen-Buch - Wilde Tiere,Elizabeth Golding,Edition Michael Fischer,WFTM,"[WD,WFTM,YBG,YBL,YBLD,YBLN1]"


,itemID
0,12
1,45274
2,10104
3,41371
4,14015


,sessionID,itemID,click,basket,order,session_count,operate
7,7,14576,1,1,0,2,0.4
8,7,17731,2,1,0,2,0.5
13,12,30277,1,0,0,3,0.1
14,12,29508,1,1,0,3,0.4
15,12,75659,1,0,0,3,0.1


# Correlation Analysis - Pearsonr Correlation Coefficient



## Pipeline
- for small data set , like only order/basket
    1. clean transaction data
    2. create pivot matrix of session and item
    3. create a peasonr correlation matrix of item and item
    4. recommend top 5 books ordered by peasonr coefficient
    
    
- for big data set, to avoid MemoryError
    1. create pivot matrix by chunks / sparse matrix
    2. create top 5 recommandations for each book and save as pickle

In [16]:
# use sparse matrix to fix MemoryError
def create_corr_sparse(transactions, value_column = 'operate'):
  # prepare data for creating sparse matrix
  session_list = list(np.sort(transactions.sessionID.unique()))
  item_list = list(np.sort(transactions.itemID.unique()))
  transactions['row'] = transactions.sessionID.apply(lambda x: session_list.index(x))
  transactions['column'] = transactions.itemID.apply(lambda x: item_list.index(x))
  display(transactions.head())

  # create sparse DataFrame
  print(f'creating coo_matrix : {len(session_list)}, {len(item_list)}')
  session_item_coo = coo_matrix((transactions.loc[:, value_column], (transactions.row.values, transactions.column.values)), shape=(len(session_list), len(item_list)))
  session_item_pivot = pd.DataFrame.sparse.from_spmatrix(session_item_coo, index = session_list, columns=item_list)

  print('computing PCC')
  compute_pcc(session_item_pivot)


# compute and save top 5 corr for each recommended book
# save format： DataFrame{itemID, pcc}
def compute_pcc(session_item_pivot):
  for itemID in recommended_books:
    pcc = []
    if itemID in session_item_pivot.columns.values:  # todo: delete later
      book = session_item_pivot.loc[:, itemID]
      for index in session_item_pivot.columns:
          pcc.append({'itemID': index, 'pcc': max(book.corr(session_item_pivot[index]), 0)})
      pcc_df = pd.DataFrame(pcc).sort_values(by='pcc', ascending=False).iloc[:5]
      #display(pcc_df)
      print('creating pickle for book %d' % itemID)
      pcc_df.to_pickle(str(itemID) + '.pickle')

In [ ]:
# This code block only suitable for small data set, so basicly do not be used!!!

# if not remove , too big to use pivot func. for order/basket/click analysis separately
def clean_transaction(transaction_original, clean_column):
    print('there are %d rows in transactions' % transaction_original.shape[0])
    
    # remove rows : order = 0
    transactions = transaction_original[transaction_original[clean_column]!= 0]
    print('run cleaning')
    print('1. after removing transactions without %s, there are %d rows in transactions' % (clean_column, transactions.shape[0]))
    
    # remove rows: unique sessionID
    clean_column_counts = transactions['sessionID'].value_counts()
    transactions['clean_column_count'] = transactions['sessionID'].apply(lambda x: clean_column_counts[x])
    transactions = transactions[transactions['clean_column_count']>1]
    print('2. after removing transactions without only one session, there are %d rows in transactions' % transactions.shape[0])
    print('there are %d books in evaluation books' % evaluation_books['itemID'].isin(transactions['itemID']).sum())
    return transactions

# use pivot , only suitable for order analysis
def create_corr_matrix(transactions):
    # create matrix : row-session column-item
    print('creating session item pivot matrix...')
    session_item_pivot = transactions.pivot(index='sessionID', columns='itemID', values='order').fillna(0)
    print('pivot created. there are %d sessions, %d books.' % (session_item_pivot.shape[0], session_item_pivot.shape[1]))
    display(session_item_pivot.head())
    
    # create correlation matrix of item to item 
    corr_matrix = session_item_pivot.corr()
    print('corelation matrix created: {}.'.format(corr_matrix.shape))
    display(corr_matrix.head())
    return corr_matrix

# use chunk to accelerate
def create_corr_chunk(transactions, value_column = 'operate'):
    # for large amount data, MemoryError
    chunk_size = 5000
    #rows = transactions.shape[0]
    # TODO: fix MemoryError
    rows = 71000
    chunks = [x for x in range(0, rows, chunk_size)]
    chunks.append(rows)
    print(chunks)
    session_item_pivot = pd.concat([transactions.iloc[ chunks[i]:chunks[i + 1] - 1 ].pivot(index='sessionID', columns='itemID', values=value_column) for i in range(0, len(chunks) - 1)]).fillna(0)
    display(session_item_pivot.shape)
    display(session_item_pivot.head())

    compute_pcc(session_item_pivot)

In [17]:
# recommendation ----- for big data set
count = 0
threhold = 0.01
#create_corr_chunk(transaction_original)
create_corr_sparse(transaction_original)
for itemID in evaluation_books['itemID']:
  if os.path.exists(str(itemID) + '.pickle'):
    print('recommend books for %d, %s, %s, %s' % (itemID, items.loc[itemID, 'title'], 
                                                    items.loc[itemID, 'main topic'], items.loc[itemID, 'subtopics']))
    recommendation = pd.read_pickle(str(itemID) + '.pickle')
    recommendation = recommendation[recommendation['pcc']>threhold]
    if recommendation.shape[0] > 0:
        recommendation_books = recommendation.merge(items, how='left', left_on='itemID', right_index=True)
        display(recommendation_books)
        count += 1
print('there are %d books being recommaned' % count)

,sessionID,itemID,click,basket,order,session_count,operate,row,column
7,7,14576,1,1,0,2,0.4,0,2621
8,7,17731,2,1,0,2,0.5,0,3194
13,12,30277,1,0,0,3,0.1,1,5471
14,12,29508,1,1,0,3,0.4,1,5341
15,12,75659,1,0,0,3,0.1,1,13858


creating pickle for book 26662
creating pickle for book 71784
creating pickle for book 49279
creating pickle for book 77956
creating pickle for book 24747
creating pickle for book 28844
creating pickle for book 4272
creating pickle for book 67776
creating pickle for book 2244
creating pickle for book 45270
creating pickle for book 6359
creating pickle for book 6363
creating pickle for book 47336
creating pickle for book 10482
creating pickle for book 51443
creating pickle for book 2319
creating pickle for book 12564
creating pickle for book 39201
creating pickle for book 63796
creating pickle for book 8516
creating pickle for book 22868
creating pickle for book 59750
creating pickle for book 6530
creating pickle for book 39308
creating pickle for book 51599
creating pickle for book 41371
creating pickle for book 45487
creating pickle for book 14782
creating pickle for book 468
creating pickle for book 2579
creating pickle for book 53793
creating pickle for book 53838
creating pickle fo

,itemID,pcc,title,author,publisher,main topic,subtopics
7492,41371,1.000000,Dryadenhain & Dschinnenzauber (Märchenanthologie),"Philipp Busch, Jonathan Driedger, Lena Kalupne...",Ulisses Spiel & Medien,FM,"[FMB,WDHW]"
5348,29539,0.707097,Die Siebenwindküste Albernia & der Windhag,Daniel Simon Richter,Ulisses Spiel & Medien,WDHW,"[FMB,WDHW]"
9998,54689,0.283437,Aventurischer Almanach Hardcover,"Florian Don-Schauen, Daniel Simon Richter",Ulisses Spiel & Medien,FM,[WDHW]


recommend books for 56794, Damaris (Band 2): Der Ring des Fürsten, FMB, [FN]


,itemID,pcc,title,author,publisher,main topic,subtopics
10386,56794,1.000000,Damaris (Band 2): Der Ring des Fürsten,C. M. Spoerri,Sternensand Verlag,FMB,[FN]
383,2121,0.743537,Damaris (Band 1): Der Greifenorden von Chakas,C. M. Spoerri,Sternensand Verlag,FMB,[]
8080,44420,0.715345,Damaris: Das Vermächtnis der Wüstenzwerge,C. M. Spoerri,Sternensand Verlag,FMB,[FN]
8599,47195,0.455819,Die Brücke der Gezeiten 5,David Hair,Blanvalet Taschenbuchverl,FMB,[FYT]
4552,25168,0.432410,Die Brücke der Gezeiten 2,David Hair,Penhaligon,FMB,[FYT]


...

recommend books for 14782, Flügel aus Papier, YXZW, [5AL,1DTP,YFCW,YFT,YXZW,3MPBGJ-DE-H,3MPBL]


,itemID,pcc,title,author,publisher,main topic,subtopics
2660,14782,1.000000,Flügel aus Papier,Marcin Szczygielski,FISCHER KJB,YXZW,"[5AL,1DTP,YFCW,YFT,YXZW,3MPBGJ-DE-H,3MPBL]"
312,1683,0.730283,Alabama Moon,Watt Key,Oetinger Taschenbuch GmbH,YFC,[5AL]
4790,26448,0.102398,Die Zugmaus,Uwe Timm,dtv Verlagsgesellschaft,YXHB,"[5AH,5AJ,1DDF-FR-LAA,1DDU-GB-E,1DFG-DE-TBC,1DF..."


recommend books for 20399, Das Drachentor, YFH, [5AP,FMB,YFH]


,itemID,pcc,title,author,publisher,main topic,subtopics
3659,20399,1.000000,Das Drachentor,Jenny-Mai Nuyen,cbt,YFH,"[5AP,FMB,YFH]"
5534,30656,0.953555,Nijura - Das Erbe der Elfenkrone,Jenny-Mai Nuyen,cbt,FMB,"[5AP,FMB,YFH]"
14213,77653,0.757596,Die Töchter von Ilian,Jenny-Mai Nuyen,FISCHER TOR,FMB,"[5JA,FBA,FYT]"
4569,25233,0.147225,"Kalt wie Schnee, hart wie Eisen",Jenny-Mai Nuyen,cbt,YFH,"[5AQ,5AX,FMB,YFH]"
792,4310,0.117807,Schattenengel Buch 02 - Himmel,Sara Brandt,Eisermann,FMB,"[FB,FMB,FMR,FN]"


there are 118 books being recommaned


In [ ]:
# recommendation----for small dataset
count = 0
threhold = 0.01
clean_column = 'order'
corr_matrix = create_corr_matrix(clean_transaction(transaction_original, clean_column))
recommend_books_ID = corr_matrix.columns.values
for index, itemID in evaluation_books.itertuples():
    if itemID in recommend_books_ID:
        print('recommend books for %d, %s, %s, %s' % (itemID, items.loc[itemID, 'title'], 
                                                      items.loc[itemID, 'main topic'], items.loc[itemID, 'subtopics']))
        recommendation = corr_matrix[itemID].sort_values(ascending=False)[1:6] # [0] is the book itself
        recommendation = recommendation[recommendation>threhold]
        if recommendation.shape[0] > 0:
            recommendation.name = 'pearson'
            recommendation_books = recommendation.to_frame().merge(items, how='left', left_index=True, right_index=True)
            display(recommendation_books)
            count += 1
print('there are %d books being recommaned' % count)
        

there are 129501 rows in transactions
run cleaning
1. after removing transactions without order, there are 6693 rows in transactions
2. after removing transactions without only one session, there are 5813 rows in transactions
there are 51 books in evaluation books
creating session item pivot matrix...
pivot created. there are 2152 sessions, 2770 books.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


itemID,54,68,125,194,197,245,249,283,366,367,369,422,449,468,480,513,616,651,672,678,685,699,737,748,762,779,782,798,880,886,1000,1083,1122,1136,1154,1169,1177,1193,1220,1252,...,78108,78240,78258,78284,78303,78304,78316,78380,78393,78397,78471,78489,78518,78532,78553,78570,78577,78594,78622,78643,78687,78743,78761,78771,78781,78782,78796,78809,78834,78837,78860,78864,78885,78893,78916,78961,78966,78996,79044,79066
sessionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
824,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


corelation matrix created: (2770, 2770).


itemID,54,68,125,194,197,245,249,283,366,367,369,422,449,468,480,513,616,651,672,678,685,699,737,748,762,779,782,798,880,886,1000,1083,1122,1136,1154,1169,1177,1193,1220,1252,...,78108,78240,78258,78284,78303,78304,78316,78380,78393,78397,78471,78489,78518,78532,78553,78570,78577,78594,78622,78643,78687,78743,78761,78771,78781,78782,78796,78809,78834,78837,78860,78864,78885,78893,78916,78961,78966,78996,79044,79066
itemID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
54,1.000000,-0.000465,-0.000806,-0.000806,-0.000658,-0.000930,-0.000465,-0.000658,-0.000658,-0.000465,-0.000465,-0.000658,-0.000658,-0.000465,-0.000465,-0.000465,-0.000806,-0.000465,-0.000465,-0.000465,-0.000806,-0.000465,-0.000465,-0.000465,-0.000658,-0.000465,-0.000930,-0.000930,-0.000465,-0.000658,-0.000465,-0.000465,-0.000658,-0.000930,-0.000465,-0.000465,-0.000465,-0.001317,-0.000658,-0.000465,...,-0.000465,-0.000465,-0.001140,-0.000465,-0.000465,-0.000465,-0.000465,-0.000465,-0.000658,-0.000465,-0.000465,-0.000624,-0.000806,-0.000930,-0.000465,-0.000465,-0.000465,-0.000465,-0.000930,-0.000806,-0.000465,-0.000465,-0.000658,-0.000806,-0.000465,-0.000465,-0.000465,-0.000465,-0.000658,-0.001681,-0.000465,-0.000658,-0.000465,-0.000465,-0.000465,-0.000465,-0.000465,-0.000806,-0.000465,-0.000465
68,-0.000465,1.000000,-0.000806,-0.000806,-0.000658,-0.000930,-0.000465,-0.000658,-0.000658,-0.000465,-0.000465,-0.000658,-0.000658,-0.000465,-0.000465,-0.000465,-0.000806,-0.000465,-0.000465,-0.000465,-0.000806,-0.000465,-0.000465,-0.000465,-0.000658,-0.000465,-0.000930,-0.000930,-0.000465,-0.000658,-0.000465,-0.000465,-0.000658,-0.000930,-0.000465,-0.000465,-0.000465,-0.001317,-0.000658,-0.000465,...,-0.000465,-0.000465,-0.001140,-0.000465,-0.000465,-0.000465,-0.000465,-0.000465,-0.000658,-0.000465,-0.000465,-0.000624,-0.000806,-0.000930,-0.000465,-0.000465,-0.000465,-0.000465,-0.000930,-0.000806,-0.000465,-0.000465,-0.000658,-0.000806,-0.000465,-0.000465,-0.000465,-0.000465,-0.000658,-0.001681,-0.000465,-0.000658,-0.000465,-0.000465,-0.000465,-0.000465,-0.000465,-0.000806,-0.000465,-0.000465
125,-0.000806,-0.000806,1.000000,-0.001396,-0.001140,-0.001612,-0.000806,-0.001140,-0.001140,-0.000806,-0.000806,-0.001140,-0.001140,-0.000806,-0.000806,-0.000806,-0.001396,-0.000806,-0.000806,-0.000806,-0.001396,-0.000806,-0.000806,-0.000806,-0.001140,-0.000806,-0.001612,-0.001612,-0.000806,-0.001140,-0.000806,-0.000806,-0.001140,-0.001612,-0.000806,-0.000806,-0.000806,-0.002282,-0.001140,-0.000806,...,-0.000806,-0.000806,-0.001976,-0.000806,-0.000806,-0.000806,-0.000806,-0.000806,-0.001140,-0.000806,-0.000806,-0.001081,-0.001396,-0.001612,-0.000806,-0.000806,-0.000806,-0.000806,-0.001612,-0.001396,-0.000806,-0.000806,-0.001140,-0.001396,-0.000806,-0.000806,-0.000806,-0.000806,-0.001140,-0.002913,-0.000806,-0.001140,-0.000806,-0.000806,-0.000806,-0.000806,-0.000806,-0.001396,-0.000806,-0.000806
194,-0.000806,-0.000806,-0.001396,1.000000,-0.001140,-0.001612,-0.000806,-0.001140,-0.001140,-0.000806,-0.000806,-0.001140,-0.001140,-0.000806,-0.000806,-0.000806,-0.001396,-0.000806,-0.000806,-0.000806,-0.001396,-0.000806,-0.000806,-0.000806,-0.001140,-0.000806,-0.001612,-0.001612,-0.000806,-0.001140,-0.000806,-0.000806,-0.001140,-0.001612,-0.000806,-0.000806,-0.000806,-0.002282,-0.001140,-0.000806,...,-0.000806,-0.000806,-0.001976,-0.000806,-0.000806,-0.000806,-0.000806,-0.000806,-0.001140,-0.000806,-0.000806,-0.001081,-0.001396,-0.001612,-0.000806,-0.000806,-0.000806,-0.000806,-0.001612,-0.001396,-0.000806,-0.000806,-0.001140,-0.001396,-0.000806,-0.000806,-0.000806,-0.000806,-0.001140,-0.002913,-0.000806,-0.001140,-0.000806,-0.000806,-0.000806,-0.000806,-0.000806,-0.001396,-0.000806,-0.000806
197,-0.000658,-0.000658,-0.001140,-0.001140,1.000000,-0.001316,-0.000658,-0.000930,-0.000930,-0.000658,-0.000658,-0.000930,-0.000930,-0.000658,-0.000658,-0.000658,-0.001140,-0.000658,-0.000658,-0.000658,-0.001140,-0.000658,-0.000658,-0.000658,-0.000930,-0.000658,-0.0

...

recommend books for 20399, Das Drachentor, YFH, [5AP,FMB,YFH]


,pearson,title,author,publisher,main topic,subtopics
itemID,,,,,,
77653,1.0,Die Töchter von Ilian,Jenny-Mai Nuyen,FISCHER TOR,FMB,"[5JA,FBA,FYT]"
30656,1.0,Nijura - Das Erbe der Elfenkrone,Jenny-Mai Nuyen,cbt,FMB,"[5AP,FMB,YFH]"


recommend books for 40812, Die Chaosschwestern gegen den Rest der Welt, YFB, [5AL,YFN,YFQ]


,pearson,title,author,publisher,main topic,subtopics
itemID,,,,,,
48371,0.706942,Die Chaosschwestern sind die Größten,Dagmar H. Mueller,cbj,YFB,"[5AL,YFN,YFQ]"
4833,0.332713,Das Lied von Eis und Feuer 06. Die Königin der...,George R. R. Martin,Blanvalet Taschenbuchverl,FYT,"[FMB,FYT]"


there are 51 books being recommaned


In [13]:
np.sort(transaction_original.sessionID.unique())

array([     7,     12,     13, ..., 279344, 279350, 279353])